# Advanced Queries with SQLAlchemy

## Overview

This topic covers advanced SQLAlchemy querying techniques that go beyond basic CRUD operations. You'll learn about complex query patterns, advanced filtering, query optimization, and sophisticated data retrieval strategies.

## Learning Objectives

By the end of this topic, you will be able to:

1. **Write complex queries** with multiple joins and advanced filtering
2. **Use advanced SQL functions** and expressions in SQLAlchemy
3. **Implement query optimization** techniques for better performance
4. **Handle complex data relationships** with sophisticated query patterns
5. **Use advanced query features** like CTEs, window functions, and analytical queries

## Prerequisites

- Complete understanding of SQLAlchemy ORM basics
- Familiarity with model relationships
- Basic knowledge of SQL concepts
- Understanding of database indexing

Let's dive into advanced querying techniques!


## 1. Setup and Model Definition

Let's start with a comprehensive e-commerce domain that will demonstrate advanced querying techniques:


In [ ]:
# Setup and model definition for advanced queries
from sqlalchemy import create_engine, Column, Integer, String, DateTime, ForeignKey, Text, Boolean, Float, Index, Date, Time, Enum
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship, joinedload, subqueryload, selectinload
from sqlalchemy import func, and_, or_, not_, desc, asc, case, cast, extract, distinct
from sqlalchemy.sql import text
from datetime import datetime, date, timedelta
import enum

# Create database engine
engine = create_engine('sqlite:///advanced_queries.db', echo=True)
Base = declarative_base()
Session = sessionmaker(bind=engine)

# Enums for better data integrity
class OrderStatus(enum.Enum):
    PENDING = "pending"
    PROCESSING = "processing"
    SHIPPED = "shipped"
    DELIVERED = "delivered"
    CANCELLED = "cancelled"

class PaymentStatus(enum.Enum):
    PENDING = "pending"
    PAID = "paid"
    FAILED = "failed"
    REFUNDED = "refunded"

# Comprehensive e-commerce models
class Customer(Base):
    __tablename__ = 'customers'
    
    id = Column(Integer, primary_key=True)
    first_name = Column(String(50), nullable=False, index=True)
    last_name = Column(String(50), nullable=False, index=True)
    email = Column(String(100), unique=True, nullable=False, index=True)
    phone = Column(String(20))
    date_of_birth = Column(Date)
    registration_date = Column(DateTime, default=datetime.utcnow, index=True)
    is_vip = Column(Boolean, default=False)
    total_spent = Column(Float, default=0.0)
    
    # Relationships
    orders = relationship("Order", back_populates="customer", cascade="all, delete-orphan")
    addresses = relationship("Address", back_populates="customer", cascade="all, delete-orphan")
    reviews = relationship("Review", back_populates="customer", cascade="all, delete-orphan")
    
    @property
    def full_name(self):
        return f"{self.first_name} {self.last_name}"
    
    def __repr__(self):
        return f"<Customer(name='{self.full_name}', email='{self.email}')>"

class Address(Base):
    __tablename__ = 'addresses'
    
    id = Column(Integer, primary_key=True)
    street = Column(String(200), nullable=False)
    city = Column(String(100), nullable=False, index=True)
    state = Column(String(50), nullable=False, index=True)
    zip_code = Column(String(10), nullable=False)
    country = Column(String(50), default='USA')
    is_primary = Column(Boolean, default=False)
    address_type = Column(String(20), default='shipping')  # shipping, billing
    
    # Foreign keys
    customer_id = Column(Integer, ForeignKey('customers.id'), nullable=False)
    
    # Relationships
    customer = relationship("Customer", back_populates="addresses")
    orders = relationship("Order", back_populates="shipping_address")
    
    def __repr__(self):
        return f"<Address(city='{self.city}', state='{self.state}')>"

class Category(Base):
    __tablename__ = 'categories'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False, unique=True, index=True)
    description = Column(Text)
    parent_id = Column(Integer, ForeignKey('categories.id'))
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Self-referential relationship
    parent = relationship("Category", remote_side=[id], backref="children")
    products = relationship("Product", back_populates="category")
    
    def __repr__(self):
        return f"<Category(name='{self.name}')>"

class Product(Base):
    __tablename__ = 'products'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(200), nullable=False, index=True)
    description = Column(Text)
    sku = Column(String(50), unique=True, nullable=False, index=True)
    price = Column(Float, nullable=False, index=True)
    cost = Column(Float)
    weight = Column(Float)
    dimensions = Column(String(50))  # "LxWxH"
    stock_quantity = Column(Integer, default=0, index=True)
    min_stock_level = Column(Integer, default=5)
    is_active = Column(Boolean, default=True, index=True)
    is_featured = Column(Boolean, default=False)
    created_at = Column(DateTime, default=datetime.utcnow, index=True)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)
    
    # Foreign keys
    category_id = Column(Integer, ForeignKey('categories.id'), nullable=False)
    
    # Relationships
    category = relationship("Category", back_populates="products")
    order_items = relationship("OrderItem", back_populates="product", cascade="all, delete-orphan")
    reviews = relationship("Review", back_populates="product", cascade="all, delete-orphan")
    inventory_movements = relationship("InventoryMovement", back_populates="product", cascade="all, delete-orphan")
    
    @property
    def profit_margin(self):
        if self.cost and self.price:
            return ((self.price - self.cost) / self.price) * 100
        return 0
    
    def __repr__(self):
        return f"<Product(name='{self.name}', price={self.price})>"

class Order(Base):
    __tablename__ = 'orders'
    
    id = Column(Integer, primary_key=True)
    order_number = Column(String(50), unique=True, nullable=False, index=True)
    order_date = Column(DateTime, default=datetime.utcnow, index=True)
    status = Column(Enum(OrderStatus), default=OrderStatus.PENDING, index=True)
    subtotal = Column(Float, nullable=False)
    tax_amount = Column(Float, default=0.0)
    shipping_cost = Column(Float, default=0.0)
    discount_amount = Column(Float, default=0.0)
    total_amount = Column(Float, nullable=False, index=True)
    notes = Column(Text)
    
    # Foreign keys
    customer_id = Column(Integer, ForeignKey('customers.id'), nullable=False)
    shipping_address_id = Column(Integer, ForeignKey('addresses.id'))
    
    # Relationships
    customer = relationship("Customer", back_populates="orders")
    shipping_address = relationship("Address", back_populates="orders")
    order_items = relationship("OrderItem", back_populates="order", cascade="all, delete-orphan")
    payments = relationship("Payment", back_populates="order", cascade="all, delete-orphan")
    
    def __repr__(self):
        return f"<Order(number='{self.order_number}', total={self.total_amount})>"

class OrderItem(Base):
    __tablename__ = 'order_items'
    
    id = Column(Integer, primary_key=True)
    quantity = Column(Integer, nullable=False)
    unit_price = Column(Float, nullable=False)
    discount_percent = Column(Float, default=0.0)
    total_price = Column(Float, nullable=False)
    
    # Foreign keys
    order_id = Column(Integer, ForeignKey('orders.id'), nullable=False)
    product_id = Column(Integer, ForeignKey('products.id'), nullable=False)
    
    # Relationships
    order = relationship("Order", back_populates="order_items")
    product = relationship("Product", back_populates="order_items")
    
    def __repr__(self):
        return f"<OrderItem(product_id={self.product_id}, qty={self.quantity})>"

class Payment(Base):
    __tablename__ = 'payments'
    
    id = Column(Integer, primary_key=True)
    amount = Column(Float, nullable=False)
    payment_method = Column(String(50), nullable=False)  # credit_card, paypal, etc.
    status = Column(Enum(PaymentStatus), default=PaymentStatus.PENDING, index=True)
    transaction_id = Column(String(100), unique=True)
    payment_date = Column(DateTime, default=datetime.utcnow, index=True)
    failure_reason = Column(Text)
    
    # Foreign keys
    order_id = Column(Integer, ForeignKey('orders.id'), nullable=False)
    
    # Relationships
    order = relationship("Order", back_populates="payments")
    
    def __repr__(self):
        return f"<Payment(amount={self.amount}, status={self.status.value})>"

class Review(Base):
    __tablename__ = 'reviews'
    
    id = Column(Integer, primary_key=True)
    rating = Column(Integer, nullable=False)  # 1-5 stars
    title = Column(String(200))
    comment = Column(Text)
    is_verified_purchase = Column(Boolean, default=False)
    helpful_votes = Column(Integer, default=0)
    created_at = Column(DateTime, default=datetime.utcnow, index=True)
    
    # Foreign keys
    product_id = Column(Integer, ForeignKey('products.id'), nullable=False)
    customer_id = Column(Integer, ForeignKey('customers.id'), nullable=False)
    
    # Relationships
    product = relationship("Product", back_populates="reviews")
    customer = relationship("Customer", back_populates="reviews")
    
    def __repr__(self):
        return f"<Review(rating={self.rating}, product_id={self.product_id})>"

class InventoryMovement(Base):
    __tablename__ = 'inventory_movements'
    
    id = Column(Integer, primary_key=True)
    movement_type = Column(String(20), nullable=False)  # in, out, adjustment
    quantity = Column(Integer, nullable=False)
    reason = Column(String(100))
    reference_number = Column(String(50))  # order number, PO number, etc.
    created_at = Column(DateTime, default=datetime.utcnow, index=True)
    
    # Foreign keys
    product_id = Column(Integer, ForeignKey('products.id'), nullable=False)
    
    # Relationships
    product = relationship("Product", back_populates="inventory_movements")
    
    def __repr__(self):
        return f"<InventoryMovement(type='{self.movement_type}', qty={self.quantity})>"

# Create indexes for performance
Index('idx_customers_name', Customer.first_name, Customer.last_name)
Index('idx_products_category_price', Product.category_id, Product.price)
Index('idx_orders_customer_date', Order.customer_id, Order.order_date)
Index('idx_order_items_order_product', OrderItem.order_id, OrderItem.product_id)
Index('idx_reviews_product_rating', Review.product_id, Review.rating)
Index('idx_inventory_product_date', InventoryMovement.product_id, InventoryMovement.created_at)

# Create tables
Base.metadata.create_all(engine)

print("✅ Advanced models created successfully!")
print("Models: Customer, Address, Category, Product, Order, OrderItem, Payment, Review, InventoryMovement")
print("Features: Enums, self-referential relationships, computed properties, comprehensive indexing")


## 2. Advanced Query Patterns

Let's explore sophisticated query patterns that demonstrate the power of SQLAlchemy:


In [ ]:
# Create sample data for advanced querying
session = Session()

# Create categories (with hierarchy)
electronics = Category(name="Electronics", description="Electronic devices and accessories")
computers = Category(name="Computers", description="Computers and laptops", parent_id=1)
smartphones = Category(name="Smartphones", description="Mobile phones", parent_id=1)
furniture = Category(name="Furniture", description="Home and office furniture")
books = Category(name="Books", description="Books and literature")

session.add_all([electronics, computers, smartphones, furniture, books])
session.commit()

# Create customers
customers_data = [
    Customer(first_name="Alice", last_name="Johnson", email="alice@example.com", phone="555-0101", 
             date_of_birth=date(1990, 5, 15), is_vip=True, total_spent=2500.0),
    Customer(first_name="Bob", last_name="Smith", email="bob@example.com", phone="555-0102",
             date_of_birth=date(1985, 8, 22), is_vip=False, total_spent=800.0),
    Customer(first_name="Carol", last_name="Davis", email="carol@example.com", phone="555-0103",
             date_of_birth=date(1992, 3, 10), is_vip=True, total_spent=3200.0),
    Customer(first_name="David", last_name="Wilson", email="david@example.com", phone="555-0104",
             date_of_birth=date(1988, 11, 5), is_vip=False, total_spent=450.0),
    Customer(first_name="Eve", last_name="Brown", email="eve@example.com", phone="555-0105",
             date_of_birth=date(1995, 7, 18), is_vip=False, total_spent=1200.0)
]

session.add_all(customers_data)
session.commit()

# Create addresses
addresses_data = [
    Address(street="123 Main St", city="New York", state="NY", zip_code="10001", 
            customer_id=1, is_primary=True),
    Address(street="456 Oak Ave", city="Los Angeles", state="CA", zip_code="90210",
            customer_id=2, is_primary=True),
    Address(street="789 Pine Rd", city="Chicago", state="IL", zip_code="60601",
            customer_id=3, is_primary=True),
    Address(street="321 Elm St", city="Houston", state="TX", zip_code="77001",
            customer_id=4, is_primary=True),
    Address(street="654 Maple Dr", city="Phoenix", state="AZ", zip_code="85001",
            customer_id=5, is_primary=True)
]

session.add_all(addresses_data)
session.commit()

# Create products
products_data = [
    Product(name="MacBook Pro 16\"", description="High-performance laptop", sku="MBP16-001", 
            price=2499.99, cost=1800.0, weight=2.0, category_id=2, stock_quantity=25, is_featured=True),
    Product(name="iPhone 15 Pro", description="Latest smartphone", sku="IPH15-001", 
            price=999.99, cost=650.0, weight=0.2, category_id=3, stock_quantity=50, is_featured=True),
    Product(name="Gaming Chair", description="Ergonomic gaming chair", sku="CHAIR-001", 
            price=299.99, cost=150.0, weight=25.0, category_id=4, stock_quantity=15),
    Product(name="Python Programming Book", description="Learn Python programming", sku="BOOK-001", 
            price=49.99, cost=20.0, weight=1.0, category_id=5, stock_quantity=100),
    Product(name="Wireless Mouse", description="Ergonomic wireless mouse", sku="MOUSE-001", 
            price=79.99, cost=30.0, weight=0.1, category_id=2, stock_quantity=200),
    Product(name="Standing Desk", description="Adjustable standing desk", sku="DESK-001", 
            price=599.99, cost=300.0, weight=50.0, category_id=4, stock_quantity=8)
]

session.add_all(products_data)
session.commit()

# Create orders with various statuses
orders_data = [
    Order(order_number="ORD-001", customer_id=1, shipping_address_id=1, status=OrderStatus.DELIVERED,
          subtotal=2499.99, tax_amount=200.0, shipping_cost=0.0, total_amount=2699.99),
    Order(order_number="ORD-002", customer_id=2, shipping_address_id=2, status=OrderStatus.SHIPPED,
          subtotal=999.99, tax_amount=80.0, shipping_cost=15.0, total_amount=1094.99),
    Order(order_number="ORD-003", customer_id=1, shipping_address_id=1, status=OrderStatus.PROCESSING,
          subtotal=299.99, tax_amount=24.0, shipping_cost=25.0, total_amount=348.99),
    Order(order_number="ORD-004", customer_id=3, shipping_address_id=3, status=OrderStatus.DELIVERED,
          subtotal=129.98, tax_amount=10.4, shipping_cost=10.0, total_amount=150.38),
    Order(order_number="ORD-005", customer_id=4, shipping_address_id=4, status=OrderStatus.CANCELLED,
          subtotal=599.99, tax_amount=48.0, shipping_cost=50.0, total_amount=697.99)
]

session.add_all(orders_data)
session.commit()

# Create order items
order_items_data = [
    OrderItem(order_id=1, product_id=1, quantity=1, unit_price=2499.99, total_price=2499.99),
    OrderItem(order_id=2, product_id=2, quantity=1, unit_price=999.99, total_price=999.99),
    OrderItem(order_id=3, product_id=3, quantity=1, unit_price=299.99, total_price=299.99),
    OrderItem(order_id=4, product_id=4, quantity=1, unit_price=49.99, total_price=49.99),
    OrderItem(order_id=4, product_id=5, quantity=1, unit_price=79.99, total_price=79.99),
    OrderItem(order_id=5, product_id=6, quantity=1, unit_price=599.99, total_price=599.99)
]

session.add_all(order_items_data)
session.commit()

# Create payments
payments_data = [
    Payment(order_id=1, amount=2699.99, payment_method="credit_card", status=PaymentStatus.PAID, 
            transaction_id="TXN-001", payment_date=datetime.utcnow() - timedelta(days=5)),
    Payment(order_id=2, amount=1094.99, payment_method="paypal", status=PaymentStatus.PAID,
            transaction_id="TXN-002", payment_date=datetime.utcnow() - timedelta(days=3)),
    Payment(order_id=3, amount=348.99, payment_method="credit_card", status=PaymentStatus.PENDING,
            transaction_id="TXN-003", payment_date=datetime.utcnow() - timedelta(days=1)),
    Payment(order_id=4, amount=150.38, payment_method="credit_card", status=PaymentStatus.PAID,
            transaction_id="TXN-004", payment_date=datetime.utcnow() - timedelta(days=7)),
    Payment(order_id=5, amount=697.99, payment_method="credit_card", status=PaymentStatus.FAILED,
            transaction_id="TXN-005", failure_reason="Insufficient funds")
]

session.add_all(payments_data)
session.commit()

# Create reviews
reviews_data = [
    Review(rating=5, title="Excellent laptop!", comment="Great performance and build quality", 
           product_id=1, customer_id=1, is_verified_purchase=True, helpful_votes=12),
    Review(rating=4, title="Good phone", comment="Nice features but expensive", 
           product_id=2, customer_id=2, is_verified_purchase=True, helpful_votes=8),
    Review(rating=5, title="Comfortable chair", comment="Very comfortable for long work sessions", 
           product_id=3, customer_id=1, is_verified_purchase=True, helpful_votes=15),
    Review(rating=4, title="Great book", comment="Well written and comprehensive", 
           product_id=4, customer_id=3, is_verified_purchase=True, helpful_votes=6),
    Review(rating=3, title="Decent mouse", comment="Works well but could be better", 
           product_id=5, customer_id=3, is_verified_purchase=True, helpful_votes=3)
]

session.add_all(reviews_data)
session.commit()

# Create inventory movements
inventory_movements_data = [
    InventoryMovement(product_id=1, movement_type="in", quantity=30, reason="Initial stock", reference_number="PO-001"),
    InventoryMovement(product_id=1, movement_type="out", quantity=1, reason="Order fulfillment", reference_number="ORD-001"),
    InventoryMovement(product_id=2, movement_type="in", quantity=60, reason="Initial stock", reference_number="PO-002"),
    InventoryMovement(product_id=2, movement_type="out", quantity=1, reason="Order fulfillment", reference_number="ORD-002"),
    InventoryMovement(product_id=3, movement_type="in", quantity=20, reason="Initial stock", reference_number="PO-003"),
    InventoryMovement(product_id=3, movement_type="out", quantity=1, reason="Order fulfillment", reference_number="ORD-003")
]

session.add_all(inventory_movements_data)
session.commit()

print("✅ Sample data created successfully!")
print(f"Categories: {session.query(Category).count()}")
print(f"Customers: {session.query(Customer).count()}")
print(f"Products: {session.query(Product).count()}")
print(f"Orders: {session.query(Order).count()}")
print(f"Reviews: {session.query(Review).count()}")
print(f"Inventory Movements: {session.query(InventoryMovement).count()}")


In [ ]:
# Advanced Query Patterns
print("=== Advanced Query Patterns ===")

# 1. Complex multi-table joins with filtering
print("\n1. Complex Multi-Table Joins:")
print("Find all delivered orders with customer details, shipping addresses, and payment status:")

delivered_orders = session.query(
    Order.order_number,
    Customer.first_name,
    Customer.last_name,
    Address.city,
    Address.state,
    Order.total_amount,
    Payment.status.label('payment_status'),
    Payment.payment_method
).join(Customer).join(Address, Order.shipping_address_id == Address.id).join(Payment).filter(
    Order.status == OrderStatus.DELIVERED
).all()

for order_num, first_name, last_name, city, state, total, payment_status, payment_method in delivered_orders:
    print(f"  {order_num}: {first_name} {last_name} from {city}, {state} - ${total} ({payment_status.value} via {payment_method})")

# 2. Advanced filtering with multiple conditions
print("\n2. Advanced Filtering with Multiple Conditions:")
print("VIP customers who spent more than $2000 and have verified purchase reviews:")

vip_customers_with_reviews = session.query(Customer).join(Review).filter(
    and_(
        Customer.is_vip == True,
        Customer.total_spent > 2000,
        Review.is_verified_purchase == True
    )
).distinct().all()

for customer in vip_customers_with_reviews:
    review_count = session.query(func.count(Review.id)).filter(Review.customer_id == customer.id).scalar()
    print(f"  {customer.full_name}: ${customer.total_spent:.2f} total, {review_count} verified reviews")

# 3. Using CASE statements for conditional logic
print("\n3. CASE Statements for Conditional Logic:")
print("Customer spending categories based on total amount:")

spending_categories = session.query(
    Customer.first_name,
    Customer.last_name,
    Customer.total_spent,
    case(
        (Customer.total_spent >= 2000, 'High Value'),
        (Customer.total_spent >= 1000, 'Medium Value'),
        (Customer.total_spent >= 500, 'Low Value'),
        else_='New Customer'
    ).label('spending_category')
).all()

for first_name, last_name, total_spent, category in spending_categories:
    print(f"  {first_name} {last_name}: ${total_spent:.2f} - {category}")

# 4. Complex aggregations with grouping
print("\n4. Complex Aggregations with Grouping:")
print("Product performance metrics by category:")

product_metrics = session.query(
    Category.name.label('category_name'),
    func.count(Product.id).label('product_count'),
    func.avg(Product.price).label('avg_price'),
    func.sum(Product.stock_quantity).label('total_stock'),
    func.count(Review.id).label('review_count'),
    func.avg(Review.rating).label('avg_rating')
).join(Product).outerjoin(Review).group_by(Category.id, Category.name).all()

for category, count, avg_price, stock, reviews, avg_rating in product_metrics:
    rating_str = f"{avg_rating:.1f}★" if avg_rating else "No ratings"
    print(f"  {category}: {count} products, ${avg_price:.2f} avg price, {stock} in stock, {reviews} reviews ({rating_str})")

# 5. Window functions for ranking and analytics
print("\n5. Window Functions for Ranking:")
print("Top products by revenue within each category:")

from sqlalchemy import over

product_revenue_ranking = session.query(
    Product.name,
    Category.name.label('category_name'),
    func.sum(OrderItem.total_price).label('total_revenue'),
    func.row_number().over(
        partition_by=Category.name,
        order_by=func.sum(OrderItem.total_price).desc()
    ).label('revenue_rank')
).join(Category).join(OrderItem).group_by(Product.id, Product.name, Category.name).all()

for name, category, revenue, rank in product_revenue_ranking:
    if revenue:
        print(f"  #{rank} {name} ({category}): ${revenue:.2f}")

# 6. Date/time functions and filtering
print("\n6. Date/Time Functions and Filtering:")
print("Recent orders (last 7 days) with customer age calculation:")

recent_orders = session.query(
    Order.order_number,
    Customer.first_name,
    Customer.last_name,
    extract('year', func.age(Customer.date_of_birth)).label('customer_age'),
    Order.order_date,
    Order.total_amount
).join(Customer).filter(
    Order.order_date >= datetime.utcnow() - timedelta(days=7)
).all()

for order_num, first_name, last_name, age, order_date, total in recent_orders:
    print(f"  {order_num}: {first_name} {last_name} (age {age}) - ${total} on {order_date.strftime('%Y-%m-%d')}")

# 7. Self-referential queries (category hierarchy)
print("\n7. Self-Referential Queries (Category Hierarchy):")
print("Category hierarchy with parent-child relationships:")

# Create alias for parent category
parent_category = Category.__table__.alias('parent_cat')

category_hierarchy = session.query(
    Category.name,
    Category.parent_id,
    func.coalesce(parent_category.c.name, 'Root').label('parent_name')
).outerjoin(parent_category, Category.parent_id == parent_category.c.id).all()

for name, parent_id, parent_name in category_hierarchy:
    indent = "  " if parent_id else ""
    print(f"{indent}{name} (parent: {parent_name})")

# 8. Complex subqueries with EXISTS
print("\n8. Complex Subqueries with EXISTS:")
print("Products that have been ordered but never reviewed:")

products_without_reviews = session.query(Product).filter(
    and_(
        session.query(OrderItem).filter(OrderItem.product_id == Product.id).exists(),
        not_(session.query(Review).filter(Review.product_id == Product.id).exists())
    )
).all()

for product in products_without_reviews:
    order_count = session.query(func.count(OrderItem.id)).filter(OrderItem.product_id == product.id).scalar()
    print(f"  {product.name}: {order_count} orders, 0 reviews")

print("\n" + "="*60)
print("Advanced query patterns demonstrated successfully!")
print("="*60)


## 3. Query Optimization Techniques

Let's explore advanced optimization strategies for complex queries:


In [ ]:
# Query Optimization Techniques
print("=== Query Optimization Techniques ===")

# 1. Eager loading strategies comparison
print("\n1. Eager Loading Strategies Comparison:")

# Turn off echo for cleaner performance output
engine.echo = False

import time

# N+1 problem demonstration
print("N+1 Problem (BAD):")
start_time = time.time()
orders = session.query(Order).limit(3).all()
for order in orders:
    customer_name = order.customer.full_name
    for item in order.order_items:
        product_name = item.product.name
n_plus_1_time = time.time() - start_time
print(f"  Time: {n_plus_1_time:.4f} seconds")

# Optimized with joinedload
print("Optimized with joinedload (GOOD):")
start_time = time.time()
optimized_orders = session.query(Order).options(
    joinedload(Order.customer),
    joinedload(Order.order_items).joinedload(OrderItem.product)
).limit(3).all()
for order in optimized_orders:
    customer_name = order.customer.full_name
    for item in order.order_items:
        product_name = item.product.name
joinedload_time = time.time() - start_time
print(f"  Time: {joinedload_time:.4f} seconds")

# 2. Query result caching
print("\n2. Query Result Caching:")
cache = {}

def cached_query(query_key, query_func):
    if query_key in cache:
        print(f"  Cache hit for '{query_key}'")
        return cache[query_key]
    else:
        print(f"  Cache miss for '{query_key}' - executing query")
        result = query_func()
        cache[query_key] = result
        return result

# First call - cache miss
start_time = time.time()
expensive_products = cached_query("expensive_products", 
    lambda: session.query(Product).filter(Product.price > 500).all())
first_call_time = time.time() - start_time

# Second call - cache hit
start_time = time.time()
expensive_products_cached = cached_query("expensive_products",
    lambda: session.query(Product).filter(Product.price > 500).all())
second_call_time = time.time() - start_time

print(f"  First call: {first_call_time:.4f}s")
print(f"  Cached call: {second_call_time:.4f}s")
print(f"  Cache improvement: {first_call_time/second_call_time:.1f}x faster")

# 3. Pagination for large result sets
print("\n3. Pagination for Large Result Sets:")
def paginate_products(page=1, per_page=3):
    offset = (page - 1) * per_page
    products = session.query(Product).offset(offset).limit(per_page).all()
    total_count = session.query(Product).count()
    total_pages = (total_count + per_page - 1) // per_page
    
    return products, total_count, total_pages

for page in range(1, 4):
    products, total, pages = paginate_products(page, 3)
    print(f"  Page {page}/{pages} ({len(products)} products):")
    for product in products:
        print(f"    {product.name} - ${product.price}")

# 4. Using indexes effectively
print("\n4. Index Usage Examples:")
print("  - Products by price range (indexed):")
start_time = time.time()
expensive_products = session.query(Product).filter(Product.price > 1000).all()
index_time = time.time() - start_time
print(f"    Found {len(expensive_products)} products in {index_time:.4f}s")

print("  - Customers by name (indexed):")
start_time = time.time()
johnson_customers = session.query(Customer).filter(Customer.last_name == 'Johnson').all()
name_time = time.time() - start_time
print(f"    Found {len(johnson_customers)} customers in {name_time:.4f}s")

# 5. Query plan analysis (simulation)
print("\n5. Query Plan Analysis (Simulation):")
print("Complex query with multiple joins:")

complex_query = session.query(
    Customer.first_name,
    Customer.last_name,
    func.count(Order.id).label('order_count'),
    func.sum(Order.total_amount).label('total_spent'),
    func.avg(Review.rating).label('avg_rating')
).join(Order).outerjoin(Review).group_by(Customer.id, Customer.first_name, Customer.last_name).all()

print("  Query executed successfully with:")
print(f"  - {len(complex_query)} customers analyzed")
print("  - Multiple table joins optimized")
print("  - Aggregations computed efficiently")

# 6. Bulk operations for performance
print("\n6. Bulk Operations for Performance:")
print("Bulk update example - updating product prices:")

# Simulate bulk price update
products_to_update = session.query(Product).filter(Product.category_id == 2).all()  # Electronics
print(f"  Updating prices for {len(products_to_update)} electronics products")

start_time = time.time()
for product in products_to_update:
    product.price = product.price * 1.05  # 5% increase
session.commit()
bulk_update_time = time.time() - start_time

print(f"  Bulk update completed in {bulk_update_time:.4f}s")

# 7. Connection pooling simulation
print("\n7. Connection Pooling Benefits:")
print("  - Reusing database connections")
print("  - Reducing connection overhead")
print("  - Better resource management")
print("  - Improved concurrent performance")

# Turn echo back on
engine.echo = True

print("\n" + "="*50)
print("Query optimization techniques demonstrated!")
print("="*50)


## 4. Advanced SQL Functions and Expressions

Let's explore advanced SQL functions and expressions available in SQLAlchemy:


In [ ]:
# Advanced SQL Functions and Expressions
print("=== Advanced SQL Functions and Expressions ===")

# 1. Mathematical functions
print("\n1. Mathematical Functions:")
print("Product profit margins and price analysis:")

profit_analysis = session.query(
    Product.name,
    Product.price,
    Product.cost,
    func.round((Product.price - Product.cost), 2).label('profit'),
    func.round(((Product.price - Product.cost) / Product.price * 100), 2).label('margin_percent'),
    func.power(Product.price, 0.5).label('price_sqrt')  # Square root for analysis
).filter(Product.cost.isnot(None)).all()

for name, price, cost, profit, margin, sqrt_price in profit_analysis:
    print(f"  {name}: ${price} - ${cost} = ${profit} ({margin}% margin)")

# 2. String functions
print("\n2. String Functions:")
print("Customer name analysis:")

name_analysis = session.query(
    Customer.first_name,
    Customer.last_name,
    func.upper(Customer.first_name).label('first_upper'),
    func.lower(Customer.last_name).label('last_lower'),
    func.length(Customer.first_name).label('first_name_length'),
    func.concat(Customer.first_name, ' ', Customer.last_name).label('full_name_concat')
).all()

for first, last, first_upper, last_lower, length, full_name in name_analysis:
    print(f"  {first} {last} -> {first_upper} {last_lower} (length: {length})")

# 3. Date/time functions
print("\n3. Date/Time Functions:")
print("Customer age and registration analysis:")

date_analysis = session.query(
    Customer.first_name,
    Customer.last_name,
    Customer.date_of_birth,
    Customer.registration_date,
    func.extract('year', func.age(Customer.date_of_birth)).label('age'),
    func.date_part('month', Customer.registration_date).label('reg_month'),
    func.date_trunc('month', Customer.registration_date).label('reg_month_start')
).all()

for first, last, dob, reg_date, age, reg_month, month_start in date_analysis:
    print(f"  {first} {last}: age {age}, registered in month {reg_month}")

# 4. Conditional functions
print("\n4. Conditional Functions:")
print("Product status and pricing tiers:")

conditional_analysis = session.query(
    Product.name,
    Product.price,
    Product.stock_quantity,
    case(
        (Product.price > 1000, 'Premium'),
        (Product.price > 500, 'High'),
        (Product.price > 100, 'Medium'),
        else_='Budget'
    ).label('price_tier'),
    case(
        (Product.stock_quantity > 50, 'In Stock'),
        (Product.stock_quantity > 10, 'Low Stock'),
        (Product.stock_quantity > 0, 'Very Low Stock'),
        else_='Out of Stock'
    ).label('stock_status')
).all()

for name, price, stock, tier, status in conditional_analysis:
    print(f"  {name}: ${price} ({tier}), {stock} units ({status})")

# 5. Aggregation functions with grouping
print("\n5. Advanced Aggregation Functions:")
print("Category statistics:")

category_stats = session.query(
    Category.name,
    func.count(Product.id).label('product_count'),
    func.avg(Product.price).label('avg_price'),
    func.min(Product.price).label('min_price'),
    func.max(Product.price).label('max_price'),
    func.stddev(Product.price).label('price_stddev'),
    func.variance(Product.price).label('price_variance'),
    func.sum(Product.stock_quantity).label('total_stock')
).join(Product).group_by(Category.id, Category.name).all()

for category, count, avg_price, min_price, max_price, stddev, variance, stock in category_stats:
    print(f"  {category}: {count} products, ${avg_price:.2f} avg (${min_price:.2f}-${max_price:.2f})")

# 6. Window functions
print("\n6. Window Functions:")
print("Product rankings within categories:")

from sqlalchemy import over

product_rankings = session.query(
    Product.name,
    Category.name.label('category_name'),
    Product.price,
    func.row_number().over(
        partition_by=Category.name,
        order_by=Product.price.desc()
    ).label('price_rank'),
    func.rank().over(
        partition_by=Category.name,
        order_by=Product.price.desc()
    ).label('price_rank_with_ties'),
    func.dense_rank().over(
        partition_by=Category.name,
        order_by=Product.price.desc()
    ).label('dense_rank'),
    func.percent_rank().over(
        partition_by=Category.name,
        order_by=Product.price.desc()
    ).label('percentile_rank')
).join(Category).all()

for name, category, price, row_num, rank, dense_rank, percentile in product_rankings:
    print(f"  {name} ({category}): ${price} - Row: {row_num}, Rank: {rank}, Dense: {dense_rank}, Percentile: {percentile:.2f}")

# 7. Analytical functions
print("\n7. Analytical Functions:")
print("Running totals and moving averages:")

# Customer spending analysis with running totals
customer_spending = session.query(
    Customer.first_name,
    Customer.last_name,
    Customer.total_spent,
    func.sum(Customer.total_spent).over(
        order_by=Customer.total_spent.desc()
    ).label('running_total'),
    func.lag(Customer.total_spent, 1).over(
        order_by=Customer.total_spent.desc()
    ).label('previous_spent'),
    func.lead(Customer.total_spent, 1).over(
        order_by=Customer.total_spent.desc()
    ).label('next_spent')
).all()

for first, last, spent, running, prev, next_spent in customer_spending:
    prev_str = f"${prev:.2f}" if prev else "N/A"
    next_str = f"${next_spent:.2f}" if next_spent else "N/A"
    print(f"  {first} {last}: ${spent:.2f} (running: ${running:.2f}, prev: {prev_str}, next: {next_str})")

# 8. Custom SQL expressions
print("\n8. Custom SQL Expressions:")
print("Complex calculations with raw SQL:")

custom_calculations = session.query(
    Product.name,
    Product.price,
    Product.cost,
    text("CASE WHEN cost > 0 THEN ROUND((price - cost) / cost * 100, 2) ELSE 0 END").label('markup_percent'),
    text("CASE WHEN stock_quantity < min_stock_level THEN 'REORDER' ELSE 'OK' END").label('reorder_status')
).all()

for name, price, cost, markup, reorder in custom_calculations:
    print(f"  {name}: ${price} (markup: {markup}%, status: {reorder})")

# Close the session
session.close()

print("\n" + "="*60)
print("Advanced SQL functions and expressions demonstrated!")
print("="*60)


## Summary

You've learned about:

1. **Advanced Query Patterns** - Complex multi-table joins, advanced filtering, CASE statements, and self-referential queries
2. **Query Optimization** - Eager loading strategies, caching, pagination, indexing, and bulk operations
3. **Advanced SQL Functions** - Mathematical, string, date/time, conditional, and analytical functions
4. **Window Functions** - Ranking, partitioning, and analytical queries for complex data analysis

## Key Takeaways

- **Complex Joins**: Use multiple table joins with proper filtering for comprehensive data retrieval
- **Query Optimization**: Always use eager loading to prevent N+1 problems and implement caching for expensive queries
- **Advanced Functions**: Leverage SQL functions for complex calculations and data transformations
- **Window Functions**: Powerful for rankings, running totals, and analytical queries
- **Performance**: Monitor query performance and use appropriate optimization techniques

## Best Practices

- Use eager loading (`joinedload`, `subqueryload`) to prevent N+1 queries
- Implement query result caching for expensive operations
- Use pagination for large result sets
- Add appropriate indexes for frequently queried columns
- Use bulk operations for better performance
- Profile queries to identify optimization opportunities
- Use window functions for complex analytical queries

Ready to practice? Move on to the exercise notebook!
